# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Tomás Ignacio Reyes Oyarzún

### **Link de repositorio de GitHub:** [Repo - TR](https://github.com/TomiReyes/MDS7202-TR)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
#!pip install -qqq gymnasium stable_baselines3
#!pip install -qqq swig
#!pip install -qqq gymnasium[box2d]

zsh:1: no matches found: gymnasium[box2d]


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [2]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

El ambiante Blackjack representa el juego donde se compite contra el dealer respecto a quien está más cerca del número 21 sumando sus cartas. Dentro del juego se pueden distinguir los siguientes estados: 
- Suma del número de las cartas actual del jugador.
- Carta del dealer.
- Si el jugador tiene un As usable, es decir, que valga 11 si no le hace pasar de 21, de lo contrario vale 1. 

En las acciones se ve lo siguiente: 
- Pedir cartas, para acercarse lo máximo a 21, si el jugador pasa de ese número pierde. 
- Detenerse, no pedir más cartas y ver cuanto suma el dealer para comparar. 

Y en recompesas está: 
- Si el jugador gana es +1. 
- Si el juegador pierde -1.
- Si empata con el dealer +/- 0.

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

Se que no se puede usar ciclor for o while pero al hacer algo 5000 veces la verdad no se me ocurrió como hacerlo.

In [ ]:
num_episodes = 5000
rewards = []

for _ in range(num_episodes):
    observation = env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        action = env.action_space.sample()  
        observation, reward, done, truncated, info = env.step(action)
        episode_reward += reward  

    rewards.append(episode_reward)  

average_reward = np.mean(rewards)
std_dev_reward = np.std(rewards)

print(f"Promedio de recompensa: {average_reward}")
print(f"Desviación estándar de la recompensa: {std_dev_reward}")

Promedio de recompensa: -0.4006
Desviación estándar de la recompensa: 0.8962810050425033


El promedio de las recompensas siempre se aproxima a -0.4, con una desviación estándar de aprox 0.9. Esto quiere decir que es más normal perder que ganar, lo cual hace sentido por como está hecho el juego y el propósito que tiene. El hecho de tener una desviación estándar alta (respecto al promedio) indica que sí hay casos en que se puede ganar y tener buenos premios, también da a entender que hay alta variabilidad por el factor suerte que hay más allá de la estrategia. 

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

Usando PPO, por su comportamiento con los problemas discretos

In [22]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.evaluation import evaluate_policy

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.37     |
|    ep_rew_mean     | -0.27    |
| time/              |          |
|    fps             | 4957     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.28        |
|    ep_rew_mean          | -0.46       |
| time/                   |             |
|    fps                  | 3362        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.018564414 |
|    clip_fraction        | 0.359       |
|    clip_range           | 0.2         |
|    entropy_loss   

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [23]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5000)

print(f"Recompensa promedio: {mean_reward}")
print(f"Desviación estándar de la recompensa: {std_reward}")

Recompensa promedio: -0.1152
Desviación estándar de la recompensa: 0.9605878200352116


Se observa una disminución considerable en el promedio de la recompensa, pasando de un aproximado -0.4 a un -0.1, donde sigue siendo más probable perder en el largo plazo, pero por el alto valor del desviación estándar sigue existiendo alta variabilidad, dandole sentido al factor suerte y las probabilidades de obtener recompensas en el juego. 

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
def obtener_accion(model, estado):
    accion, _ = model.predict([estado], deterministic=True)
    return accion[0]  #

estado = env.reset()[0]
print(f"Clase del estado: {type(estado)}, Estado de ejemplo: {estado}")

Clase del estado: <class 'numpy.ndarray'>, Estado de ejemplo: [19 10  0]


In [ ]:
estado1 = (6, 7, False)
accion1 = obtener_accion(model, estado1)
print(f"Acción para el escenario 1: {accion1}")

estado2 = (19, 3, True)
accion2 = obtener_accion(model, estado2)
print(f"Acción para el escenario 2: {accion2}")

Acción para el escenario 1: 1
Acción para el escenario 2: 0


Son coherentes las desiciones que se tomaron dado que pidió carta cuando tenía un 6, que es una mano baja y con altas probabilidades de perder, y no pidió carta cuando tenía un 19 que es una buena mano y es muy probable que pierde si pide otra carta.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [39]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [40]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

LunarLander es un juego de aterrizar una nave de manera correcta en el espacio que corresponde. Dentro del juego se pueden distinguir los siguientes estados: 
- Posición de la nave en el eje x y eje y.
- Velocidades en x e y.
- Ángulo de orientación y velocidad angular. 
- Indicador si las patas de aterrizaje izquierda y derecha tocaron el suelo. 

Considerando el modo continuo, las acciones son: 
- Impulso del motor vertical 
- Impulso de motores horizontales. 

Y en recompesas está: 
- Recompensa por acercarse a la plataforma y disminuir la velocidad.
- +10 puntos por cada pata que toque el suelo.
- -0.3 por uso del motor principal, -0.03 por uso de motores horizontales.
- +100 puntos aterrizaje exitoso.
- -100 puntos por chocar. 

En este ambiente existen más escenarios, estados, acciones y recompensas que en el blackjack, porque el juego de azar es un ambiente discreto mientras que LunarLander trata de un ambiente continuo con más desiciones que tomar.

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [ ]:
num_episodes = 10
rewards = []

for _ in range(num_episodes):
    observation = env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        action = env.action_space.sample()
        observation, reward, done, _, _ = env.step(action)
        episode_reward += reward  
    rewards.append(episode_reward) 

average_reward = np.mean(rewards)
std_dev_reward = np.std(rewards)

print(f"Promedio de recompensa: {average_reward}")
print(f"Desviación estándar de la recompensa: {std_dev_reward}")

Promedio de recompensa: -236.68379621334884
Desviación estándar de la recompensa: 101.19090565970589


Si se realiza de manera aleatoria, no tiene sentido dado que se penaliza bastante el chocar o tomar desiciones al azar. Esto se explica con el promedio de recompensa que se aproxima a -250, con una desviación estándar cercana a 100, quiere decir que en la mayoría de los casos se pierde, esto porque se requiere tener más sentido en las desiciones al tratar de un juego continuo.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
model = PPO("MlpPolicy", env, verbose=1)

model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 106      |
|    ep_rew_mean     | -153     |
| time/              |          |
|    fps             | 3994     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 108         |
|    ep_rew_mean          | -166        |
| time/                   |             |
|    fps                  | 3017        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004465096 |
|    clip_fraction        | 0.0334      |
|    clip_range           | 0.2         |
|    entropy_loss   

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Recompensa promedio: {mean_reward}")
print(f"Desviación estándar de la recompensa: {std_reward}")

Recompensa promedio: -155.72273620029446
Desviación estándar de la recompensa: 96.92851277984472


Es mejor que el escenario baseline pero sigue siendo negativo, al tratar de un ejercicio continuo puede que el modelo necesite más tiempo para aprender de mejor manera. De todos modos, aumentó la recompensa de -250 a -150 aprox. Con la misma desviación estándar, por lo que el juego aún tiene variabilidad que afecta en su desempeño.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
learning_rate = 5e-4  
batch_size = 32       
total_timesteps = 100000 

model = PPO("MlpPolicy", env, verbose=1, learning_rate=learning_rate, batch_size=batch_size)

model.learn(total_timesteps=total_timesteps)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Recompensa promedio: {mean_reward}")
print(f"Desviación estándar de la recompensa: {std_reward}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 117      |
|    ep_rew_mean     | -281     |
| time/              |          |
|    fps             | 3953     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 111         |
|    ep_rew_mean          | -263        |
| time/                   |             |
|    fps                  | 2581        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007455999 |
|    clip_fraction        | 0.0477      |
|    clip_range           | 0.2         |
|    entropy_loss   

/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Recompensa promedio: 120.71579263039924
Desviación estándar de la recompensa: 82.10914774619216


In [76]:
export_gif(model, n=5)

![Agent Performance GIF](agent_performance.gif)

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
#%pip install --upgrade --quiet PyPDF2


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import PyPDF2

doc_paths = ['/Users/tomasreyes/Desktop/Cosas U /Laboratorio de Programación Científica/MDS7202-1/pdfs/analysing-soccer-cluster-concepts.pdf', 
             "/Users/tomasreyes/Desktop/Cosas U /Laboratorio de Programación Científica/MDS7202-1/pdfs/Clustering_soccer_players_investigating.pdf", 
             "/Users/tomasreyes/Desktop/Cosas U /Laboratorio de Programación Científica/MDS7202-1/pdfs/ml-approach-xg.pdf"] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader1 = PyPDFLoader(doc_paths[0])
loader2 = PyPDFLoader(doc_paths[1])
loader3 = PyPDFLoader(doc_paths[2])

docs = loader1.load() + loader2.load() + loader3.load()

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) 
splits = text_splitter.split_documents(docs)

In [31]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") 
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) 
vectorstore

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", 
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Usando el contexto siguiente:\n{context}\nResponde la pregunta: {question}"
)

qa_chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=prompt_template)

rag_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=qa_chain
)


#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

Caso1: 
- Pregunta: ¿Cuál es el objetivo principal del documento sobre Expected Goals (xG)?
- Respuesta correcta: Desarrollar un modelo de Expected Goals (xG) ajustado para cada posición y jugador en fútbol, mejorando la precisión en la predicción de goles según datos de StatsBomb.

Caso2: 
- Pregunta: ¿Qué métodos de clustering se utilizan en el documento de clustering de jugadores de fútbol?
- Respuesta correcta: El documento utiliza K-means y Expectation Maximization (EM) para agrupar a jugadores según su posición, evaluando la pureza y el puntaje de silhouette.

Caso3: 
- Pregunta: ¿Qué es un "conceptor" en el contexto del análisis de juegos de fútbol?
- Respuesta correcta: Un "conceptor" es una herramienta de redes neuronales recurrentes que ayuda a identificar y predecir patrones repetitivos en los juegos de fútbol simulados.

In [29]:
respuesta1 = rag_chain.invoke({"query": "¿Cuál es el objetivo principal del documento sobre Expected Goals (xG)?"})
print(respuesta1)

{'query': '¿Cuál es el objetivo principal del documento sobre Expected Goals (xG)?', 'result': 'El objetivo principal del documento es crear y aplicar un modelo de Expected Goals (xG) "desde cero" para evaluar el rendimiento de equipos e individuos.\n'}


In [17]:
respuesta2 = rag_chain.invoke({"query": "¿Qué métodos de clustering se utilizan en el documento de clustering de jugadores de fútbol?"})
print(respuesta2)

{'query': '¿Qué métodos de clustering se utilizan en el documento de clustering de jugadores de fútbol?', 'result': 'El documento utiliza dos métodos de clustering: K-means y Expectation Maximization (EM).\n'}


In [18]:
respuesta3 = rag_chain.run({"query": "¿Qué es un 'conceptor' en el contexto del análisis de juegos de fútbol?"})
print(respuesta3)

En el contexto del análisis de juegos de fútbol, un "conceptor" es una matriz de baja dimensionalidad (una variedad de baja dimensión) que describe trayectorias a través de un espacio de estados de alta dimensionalidad.  Estos permiten predicciones específicas de la situación a partir de la misma red neuronal recurrente.  En esencia, los conceptores restringen la dinámica de la red neuronal a un subespacio lineal característico de un patrón específico (como un movimiento en el juego), permitiendo identificar y analizar movimientos como "pase" o "regate" sin necesidad de definirlos previamente.  Permiten segmentar el juego en secuencias de situaciones aprendidas de forma no supervisada y predecir el futuro cercano de cada situación.



#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

* Tamaño del chunk: El tamaño del chunk afecta en la vectorización de los documentos por lo que si el tamaño del chunk es menor, afecta en la eficiencia del modelo. Ahora pasando a las respuestas: con chunks más grandes (1000 en este caso) la respuesta es más larga aunque las dos tienen información similar, la respuesta con chunk de 500 es más precisa, por lo tanto, es mejor. Comparando con chunks de menor tamaño (100 en este caso) la respuesta es más corta y pierde harta información, por lo que no es correcta o está muy incompleta.

    - Chunk de 1000: 'El objetivo principal del documento es crear y aplicar un modelo de Expected Goals (xG) "desde cero" para predecir xG,  desarrollar métricas xG ajustadas por posición y jugador, y proporcionar ventajas competitivas a la industria y mejoras en el conocimiento académico.  Esto incluye la creación de un modelo confiable y robusto que pueda predecir goles con precisión, evaluar la calidad del remate de los jugadores y lograr una correlación positiva significativa con los proveedores de xG de la industria.\n'

    - Chunk de 500 (utilizado): 'El objetivo principal del documento es crear y aplicar un modelo de Expected Goals (xG) "desde cero", utilizando características nuevas y altamente informativas para predecir los valores de xG.  También busca proporcionar ventajas competitivas a la industria y mejorar el conocimiento académico a través de la publicación de un nuevo modelo de xG ajustado por posición y jugador.\n'

    - Chunk de 100: El objetivo principal del documento es crear y aplicar un modelo de Expected Goals (xG) "desde cero" para evaluar el rendimiento de equipos e individuos


* Cantidad de chunks recuperados: La cantidad de chunks recuperados afecta en la complejidad de la respuesta dado que define cuantos fragmentos se van a utilizar para realizar la respuesta. Con más chunks la respuesta debería ser más completa pero con riesgo de tener información irrelevante, con menos chunks puede que no se logre obtener la información esperada

    - 1 recuperado: 'El texto no especifica los métodos de clustering utilizados.  Solo menciona que se agrupan los jugadores en 4 y 11 clusters basándose en su posición (portero, defensa, mediocampista, atacante) y la frecuencia de eventos que realizan.  No se nombra ningún algoritmo de clustering específico (como k-means, jerárquico, etc.).\n'

    - 3 recuperados: 'El documento utiliza dos métodos de clustering: K-means y Expectation Maximization (EM)'

    - 5 recuperados: 'El documento utiliza dos métodos de clustering no supervisados: K-means y Expectation Maximization (EM).\n'


* Tipo de búsqueda: El tipo de búsqueda define como se van a seleccionar los chunks para buscar la respuesta, aquí las respuestas con 3 distintos tipos de búsqueda:

    - Similarity: 'En el contexto del análisis de juegos de fútbol, un "conceptor" es una matriz de baja dimensionalidad (una variedad de baja dimensión) que describe trayectorias a través de un espacio de estados de alta dimensionalidad.  Estos permiten predicciones específicas de la situación a partir de la misma red neuronal recurrente.  En esencia, los conceptores restringen la dinámica de la red neuronal a un subespacio lineal característico de un patrón específico (como un movimiento en el juego), permitiendo identificar y analizar movimientos como "pase" o "regate" sin necesidad de definirlos previamente.  Permiten segmentar el juego en secuencias de situaciones aprendidas de forma no supervisada y predecir el futuro cercano de cada situación.'

    - Maximal marginal relevance: 'El texto proporcionado no define explícitamente qué es un "conceptor" en el contexto del análisis de juegos de fútbol.  El título menciona "Clustering and Conceptors", sugiriendo que son una herramienta o técnica utilizada junto con el clustering (agrupamiento) para analizar partidos de fútbol, pero no se ofrece una explicación de su función o significado.'

    - Similarity Score Threshold (score_threshold = 0.5): En el contexto del análisis de juegos de fútbol, un "conceptor" se asocia con diferentes movimientos dentro del juego.  El texto no define explícitamente qué *es* un concepter, pero implica que son unidades o categorías que representan acciones o secuencias de acciones dentro de un partido.  El objetivo del estudio es identificar estos "conceptos" a partir de datos, utilizando métodos de clustering para agrupar movimientos y establecer un repertorio de los mismos que conforman el juego completo.


### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 1) 
tools = [search] 

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [26]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [ ]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react") 
print(react_prompt.template)

/opt/homebrew/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents import Tool

tools = [
    Tool(name="TavilySearch", func=search.run, description="Busca resultados en Tavily."),
    Tool(name="WikipediaQuery", func=tool.run, description="Consulta Wikipedia.")
]

agent = create_react_agent(llm, tools, react_prompt) 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "TavilySearch(tool_input: 'Union[str, dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks: 'Callbacks' = None, *, tags: 'Optional[list[str]]' = None, metadata: 'Optional[dict[str, Any]]' = None, run_name: 'Optional[str]' = None, run_id: 'Optional[uuid.UUID]' = None, config: 'Optional[RunnableConfig]' = None, tool_call_id: 'Optional[str]' = None, **kwargs: 'Any') -> 'Any' - Busca resultados en Tavily.\nWikipediaQuery(tool_input: 'Union[str, dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks: 'Callbacks' = None, *, tags: 'Optional[list[str]]' = N

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [33]:
response = agent_executor.invoke({"input": "qué equipo ganó el mundial de fútbol 2022?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find information about the winner of the 2022 FIFA World Cup.  Wikipedia is a reliable source for this type of information.

Action: WikipediaQuery
Action Input: "2022 FIFA World Cup winner"
Page: 2022 FIFA World Cup
Summary: The 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennQuestion: qué equipo ganó el mundial de fútbol 2022?
Thought:To answer this question, I need to find information about the winner of the 2022 FIFA World Cup.  Wikipedia is a reliable source for this type of information.

Action: WikipediaQuery
Action Input: "2022 FIFA World Cup winner"
Page: 2022 FIFA World Cup
Summary: The 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennQuestion: qué equipo ganó el mundial de fútbol 2022?
Thought:To answer this question, I need to find information about the winner of the 2022 FIFA World Cup.  Wikipedia is a reliable source for this type of information.

Action: Wikipedia

In [39]:
response = agent_executor.invoke({"input": "¿Cuál es el mejor restaurante en Santiago?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Para responder a la pregunta "¿Cuál es el mejor restaurante en Santiago?", necesito buscar información sobre restaurantes en Santiago y sus reseñas.  No tengo acceso a información en tiempo real, incluyendo reseñas de restaurantes, por lo que usar TavilySearch podría ser útil si tiene información sobre rankings o reseñas de restaurantes.  WikipediaQuery probablemente no sea tan útil para esta pregunta específica, ya que Wikipedia no suele tener rankings subjetivos de restaurantes.

Action: TavilySearch
Action Input: "mejores restaurantes Santiago Chile"
[{'url': 'https://santiagosecreto.com/mejores-restaurantes-santiago/', 'content': 'Es por eso que lidera la lista de los mejores restaurantes de Santiago. Nueva de Lyon 99, Providencia, Santiago de Chile. Lunes a sábado 13:00 a 23:00 hrs. 2. La Vinoteca. Santiago Secreto. La Vinoteca es tienda, restaurante y una charcutería que te entrega una experiencia completa a través de la historia d

Como se ve en las respuestas, usa Wikipedia para respuestas que tienen más contexto o que pueden ser respondidas a partir de información documentada, en cambio usa Tavily cuando la respuesta no es tan directa o cuando hay menos información objetiva del tema.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [ ]:
def agent_tool_function(query: str) -> str:
    try:
        respuesta = agent_executor.invoke({"input": query})
        return respuesta["output"]  
    except Exception as e:
        return f"Error en AgentTool: {e}"

def rag_tool_function(query: str) -> str:
    try:
        respuesta = rag_chain.run({"query": query}) 
        return respuesta
    except Exception as e:
        return f"Error en RAGTool: {e}"


#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents import Tool
from langchain.agents import initialize_agent, AgentType

tools = [
    Tool(
        name="RAGTool",
        func=rag_tool_function,
        description="Utiliza la solución RAG para responder preguntas basadas en documentos PDF."
    ),
    Tool(
        name="AgentTool",
        func=agent_tool_function,
        description="Un agente que utiliza Wikipedia y Tavily para responder preguntas."
    )
]

supervisor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [ ]:
respuesta1 = supervisor.run("¿Cuál es el objetivo principal del documento sobre Expected Goals (xG)?")
print(respuesta1)



> Entering new AgentExecutor chain...
Thought: Necesito acceder al contenido del documento sobre Expected Goals (xG) para determinar su objetivo principal.  Como no tengo acceso directo a archivos PDF, usaré AgentTool para buscar información sobre el objetivo principal de los modelos xG.

Action: AgentTool
Action Input: "¿Cuál es el objetivo principal de los modelos de Expected Goals (xG) en el fútbol?"


> Entering new AgentExecutor chain...
Thought: Para responder a la pregunta sobre el objetivo principal de los modelos xG en el fútbol, necesito buscar información sobre estos modelos.  Wikipedia sería una buena fuente para obtener una definición general y su propósito.

Action: WikipediaQuery
Action Input: "Expected Goals (xG) in football"
Page: Expected goals
Summary: In association football, expected goals (xG) is a performance metric uQuestion: ¿Cuál es el objetivo principal de los modelos de Expected Goals (xG) en el fútbol?"
Thought: Para responder a la pregunta sobre el objet

In [ ]:
respuesta2 = supervisor.run("¿Qué métodos de clustering se utilizan en el documento de clustering de jugadores de fútbol?")
print(respuesta2)



> Entering new AgentExecutor chain...
Thought: Necesito acceder al documento de clustering de jugadores de fútbol para identificar los métodos utilizados.  Como no tengo acceso a ese documento específico, debo usar una herramienta que pueda acceder a información general sobre métodos de clustering usados en análisis deportivos.  AgentTool parece la mejor opción para esto.

Action: AgentTool
Action Input: "¿Qué métodos de clustering se utilizan comúnmente en el análisis de datos de jugadores de fútbol?"


> Entering new AgentExecutor chain...
Thought: Para responder a la pregunta sobre qué métodos de clustering se utilizan comúnmente en el análisis de datos de jugadores de fútbol, necesito buscar información sobre técnicas de clustering aplicadas en el análisis deportivo.  Tanto TavilySearch como WikipediaQuery podrían ser útiles, pero WikipediaQuery podría ser más general y necesitaría una búsqueda más específica en TavilySearch si no encuentro suficiente información.  Intentaré prim

In [ ]:
respuesta3 = supervisor.run("¿Qué es un 'conceptor' en el contexto del análisis de juegos de fútbol?")
print(respuesta3)



> Entering new AgentExecutor chain...
Thought: No tengo conocimiento previo sobre el término "conceptor" en el análisis de juegos de fútbol.  Intentaré usar AgentTool para buscar información en Wikipedia y Tavily.

Action: AgentTool
Action Input: "¿Qué es un 'conceptor' en el contexto del análisis de juegos de fútbol?"


> Entering new AgentExecutor chain...
Thought: Necesito encontrar información sobre el término "conceptor" en el contexto del análisis de juegos de fútbol.  Como es un término específico del deporte, es probable que Wikipedia no tenga una entrada dedicada.  Intentaré primero con una búsqueda en Tavily, que podría tener información de blogs, foros o sitios web especializados en análisis futbolístico.

Action: TavilySearch
Action Input: "conceptor análisis juego fútbol"
[{'url': 'https://futboldelibro.com/wp-content/uploads/2019/10/MuestraModeloDeJuego.pdf', 'content': 'MODELO DE JUEGO: ESTRUCTURA, METODOLOGÍA Y APLICACIÓN PRÁCTICA. ... cursos y sabiendo la evolución y

In [ ]:
response = supervisor.run("qué equipo ganó el mundial de fútbol 2022?")
print(response)



> Entering new AgentExecutor chain...
Thought: Necesito encontrar información sobre el ganador de la Copa Mundial de la FIFA 2022.  Puedo usar AgentTool ya que tiene acceso a Wikipedia, que seguramente tendrá esta información.

Action: AgentTool
Action Input: ¿Qué equipo ganó la Copa Mundial de la FIFA 2022?


> Entering new AgentExecutor chain...
Thought: Necesito encontrar información sobre la Copa Mundial de la FIFA 2022 para determinar qué equipo la ganó.  Wikipedia es una buena fuente para este tipo de información.

Action: WikipediaQuery
Action Input: "Copa Mundial de la FIFA 2022"
Page: Copa América de Futsal
Summary: The CONMEBOL Copa América de Futsal is the main national futsaThought: The observation is incorrect; it's about Futsal, not the FIFA World Cup. I need to try again with a more specific query.

Action: WikipediaQuery
Action Input: "2022 FIFA World Cup Winner"
Page: 2022 FIFA World Cup
Summary: The 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennThough

Para las preguntas que iban dirigidas al RAG, este enfoque entregó respuestas más generales ya que buscó información en la web, por lo que no es tan específico aplicado a los documentos como se quisiera. Para la respuesta que fue enfocada al trabajo del agente, la respuesta y el trabajo por detrás fue casi lo mismo.

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

Este enfoque como es supervisor, trabaja con los distintos llms de modo de obtener una respuesta, a diferencia del router que redigire la pregunta a un llm específico y se obtiene una sola respuesta. La ventaja de este enfoque es que obtiene respuestas más robustas y con más trabajo por detrás, es decir, se espera que tengan más sentido por más que sean más resumidas en unos casos, a diferencia de la solución router que solo entrega la respuesta del llm solicitado. 

La desventaja de este enfoque son los tiempos de ejecución y el costo computacional que hay, ya que para algunas preguntas simples se puede tardar minutos por complejizar el trabajo detrás utilizando varios llms, a diferencia de la solución router, que al tratar con un llm no es necesario hacer más trabajo por detrás, por lo que la respuesta es más rápida y más eficiente. 

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [91]:
%pip install --upgrade --quiet gradio


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = supervisor.run(message)

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot Clustering Football MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot con información de ML en el fútbol", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://33b3d3e55626aba88d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Thought: I need to define "expected goals".  Since it's a statistical concept likely related to sports, I'll try AgentTool first, as it has access to Wikipedia which is a good source for definitions.

Action: AgentTool
Action Input: Define "expected goals" in soccer (football).


> Entering new AgentExecutor chain...
Thought: To define "expected goals" in soccer, I need to find a reliable source of information, such as Wikipedia or a reputable sports statistics website.  Wikipedia is a good starting point for a general definition.

Action: WikipediaQuery
Action Input: "Expected goals soccer"
Page: Expected goals
Summary: In association football, expected goals (xG) is a performance metric uQuestion: Define "expected goals" in soccer (football).
Thought:To define "expected goals" in soccer, I need to find a reliable source of information, such as Wikipedia or a reputable sports statistics website.  Wikipedia is a good starting point for a general 